In [ ]:
import os, sys
project_dir = os.path.join(os.getcwd(),'../..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

hyspeclab_dir = os.path.join(project_dir, 'HySpecLab')
if hyspeclab_dir not in sys.path:
    sys.path.append(hyspeclab_dir)

ipdl_dir = os.path.join(project_dir, 'modules/IPDL')
if ipdl_dir not in sys.path:
    sys.path.append(ipdl_dir)

import numpy as np
import torch
import config
import torch
from torch import nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from scipy.io import loadmat

dataset_path = os.path.join(config.DERMA_DATASET_DIR, 'train/')
subjects = os.listdir(dataset_path)[::12]

subject_data = []
for i in subjects:
    mat = loadmat(os.path.join(dataset_path, i))
    subject_data.append(mat['preProcessedImage'].astype(np.float64)[np.newaxis,:])

subject_data = np.nan_to_num(np.concatenate(subject_data))

In [ ]:
import scipy as sp
import scipy.linalg as splin

#############################################
# Internal functions
#############################################

def estimate_snr(Y,r_m,x):

  [L, N] = Y.shape           # L number of bands (channels), N number of pixels
  [p, N] = x.shape           # p number of endmembers (reduced dimension)
  
  P_y     = np.sum(Y**2)/float(N)
  P_x     = np.sum(x**2)/float(N) + np.sum(r_m**2)
  return 10*np.log10( (P_x - p/L*P_y)/(P_y - P_x) )


def vca(Y,R,verbose = True,snr_input = 0):
# Vertex Component Analysis
#
# Ae, indice, Yp = vca(Y,R,verbose = True,snr_input = 0)
#
# ------- Input variables -------------
#  Y - matrix with dimensions L(channels) x N(pixels)
#      each pixel is a linear mixture of R endmembers
#      signatures Y = M x s, where s = gamma x alfa
#      gamma is a illumination perturbation factor and
#      alfa are the abundance fractions of each endmember.
#  R - positive integer number of endmembers in the scene
#
# ------- Output variables -----------
# Ae     - estimated mixing matrix (endmembers signatures)
# indice - pixels that were chosen to be the most pure
# Yp     - Data matrix Y projected.   
#
# ------- Optional parameters---------
# snr_input - (float) signal to noise ratio (dB)
# v         - [True | False]
# ------------------------------------
#
# Author: Adrien Lagrange (adrien.lagrange@enseeiht.fr)
# This code is a translation of a matlab code provided by 
# Jose Nascimento (zen@isel.pt) and Jose Bioucas Dias (bioucas@lx.it.pt)
# available at http://www.lx.it.pt/~bioucas/code.htm under a non-specified Copyright (c)
# Translation of last version at 22-February-2018 (Matlab version 2.1 (7-May-2004))
#
# more details on:
# Jose M. P. Nascimento and Jose M. B. Dias 
# "Vertex Component Analysis: A Fast Algorithm to Unmix Hyperspectral Data"
# submited to IEEE Trans. Geosci. Remote Sensing, vol. .., no. .., pp. .-., 2004
# 
# 

  #############################################
  # Initializations
  #############################################
  if len(Y.shape)!=2:
    sys.exit('Input data must be of size L (number of bands i.e. channels) by N (number of pixels)')

  [L, N]=Y.shape   # L number of bands (channels), N number of pixels
       
  R = int(R)
  if (R<0 or R>L):  
    sys.exit('ENDMEMBER parameter must be integer between 1 and L')
        
  #############################################
  # SNR Estimates
  #############################################

  if snr_input==0:
    y_m = np.mean(Y,axis=1,keepdims=True)
    Y_o = Y - y_m           # data with zero-mean
    Ud  = splin.svd(np.dot(Y_o,Y_o.T)/float(N))[0][:,:R]  # computes the R-projection matrix 
    x_p = np.dot(Ud.T, Y_o)                 # project the zero-mean data onto p-subspace

    SNR = estimate_snr(Y,y_m,x_p)

  else:
    SNR = snr_input
    
  if verbose:
    print("input SNR = {}[dB]\n".format(SNR))

  SNR_th = 15 + 10*np.log10(R)
         
  #############################################
  # Choosing Projective Projection or 
  #          projection to R-1 subspace
  #############################################

  if SNR < SNR_th:
    if verbose:
      print("... Select proj. to R-1")
                
      d = R-1
      if snr_input==0: # it means that the projection is already computed
        Ud = Ud[:,:d]
      else:
        y_m = np.mean(Y,axis=1,keepdims=True)
        Y_o = Y - y_m  # data with zero-mean 
         
        Ud  = splin.svd(np.dot(Y_o,Y_o.T)/float(N))[0][:,:d]  # computes the p-projection matrix 
        x_p =  np.dot(Ud.T,Y_o)                 # project thezeros mean data onto p-subspace
                
      Yp =  np.dot(Ud,x_p[:d,:]) + y_m      # again in dimension L
                
      x = x_p[:d,:] #  x_p =  Ud.T * Y_o is on a R-dim subspace
      c = np.amax(np.sum(x**2,axis=0))**0.5
      y = np.vstack(( x, c*np.ones((1,N)) ))
  else:
    if verbose:
      print("... Select the projective proj.")
             
    d = R
    Ud  = splin.svd(np.dot(Y,Y.T)/float(N))[0][:,:d] # computes the p-projection matrix 
                
    x_p = np.dot(Ud.T,Y)
    Yp =  np.dot(Ud,x_p[:d,:])      # again in dimension L (note that x_p has no null mean)
                
    x =  np.dot(Ud.T,Y)
    u = np.mean(x,axis=1,keepdims=True)        #equivalent to  u = Ud.T * r_m
    y =  x / (np.dot(u.T,x) + 1e-16)

  #############################################
  # VCA algorithm
  #############################################

  indice = np.zeros((R),dtype=int)
  A = np.zeros((R,R))
  A[-1,0] = 1

  for i in range(R):
    w = np.random.rand(R,1);   
    f = w - np.dot(A,np.dot(splin.pinv(A),w))
    f = f / splin.norm(f)
      
    v = np.dot(f.T,y)

    indice[i] = np.argmax(np.absolute(v))
    A[:,i] = y[:,indice[i]]        # same as x(:,indice(i))

  Ae = Yp[:,indice]

  return Ae,indice,Yp

In [ ]:
data = subject_data.reshape(subject_data.shape[0]*subject_data.shape[1]*subject_data.shape[2], subject_data.shape[3])
data = data.T
Ae, indice, Yp = vca(data, 4, True, 0)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(Ae)
plt.show()

In [ ]:
indice

In [ ]:
plt.plot(data[:,indice])

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
import scipy.linalg as splin

class VCA(TransformerMixin, BaseEstimator):
    def __init__(self, n_endmembers:int, snr_input=0):
        super(VCA, self).__init__()
        self.n_endmembers = n_endmembers
        self.snr_input = snr_input

    def fit(self, X, y=None):
        '''
            Parameters
            ----------
                X : array, shape (n_samples, n_features)
                    Matrix with dimensions N (number of pixels) x L (number of  channels). Each pixel is a
                    linear mixture of R endmembers signatures Y = M x s, where s = gamma x alfa.
                    Gamma is a illumination perturbation factor and alfa are the abundance
                    fractions of each endmember

                y : any, default=None
                    Ignored. This parameter exists only for compatibility with
                    sklearn.pipeline.Pipeline.

            Returns
            -------
            self : object
                Returns the instance itself.
        '''
        X_ = X.T
        L, N = X_.shape   # L number of bands (channels), N number of pixels
        R = self.n_endmembers

        if (self.n_endmembers<0 or self.n_endmembers>L):
            raise ValueError('Number of endmemembers parameter must be integer between 1 and L')

        if self.snr_input == 0:
            y_m = np.mean(X_,axis=1,keepdims=True)
            Y_o = X_ - y_m           # data with zero-mean
            Ud  = splin.svd(np.dot(Y_o,Y_o.T)/float(N))[0][:,:R]  # computes the R-projection matrix 
            x_p = np.dot(Ud.T, Y_o)                 # project the zero-mean data onto p-subspace

            SNR = self._estimate_snr(X_,y_m,x_p)
        else:
            SNR = self.snr_input

        SNR_th = 15 + 10*np.log10(R)

        if SNR < SNR_th:
            # ... Select proj. to R-1
            d = R-1
            if self.snr_input==0: # it means that the projection is already computed
                Ud = Ud[:,:d]
            else:
                y_m = np.mean(X_,axis=1,keepdims=True)
                Y_o = X_ - y_m  # data with zero-mean 
                
                Ud  = splin.svd(np.dot(Y_o,Y_o.T)/float(N))[0][:,:d]  # computes the p-projection matrix 
                x_p =  np.dot(Ud.T,Y_o)                 # project the zeros-mean data onto p-subspace
                        
            Yp =  np.dot(Ud,x_p[:d,:]) + y_m      # again in dimension L
                        
            x = x_p[:d,:] #  x_p =  Ud.T * Y_o is on a R-dim subspace
            c = np.amax(np.sum(x**2,axis=0))**0.5
            y = np.vstack(( x, c*np.ones((1,N)) ))

        else: 
            # ... Select the projective proj.
                    
            d = R
            Ud  = splin.svd(np.dot(X_,X_.T)/float(N))[0][:,:d] # computes the p-projection matrix 
                        
            x_p = np.dot(Ud.T,X_)
            Yp =  np.dot(Ud,x_p[:d,:])      # again in dimension L (note that x_p has no null mean)
                        
            x =  np.dot(Ud.T,X_)
            u = np.mean(x,axis=1,keepdims=True)        #equivalent to  u = Ud.T * r_m
            y =  x / (np.dot(u.T,x) + 1e-16)

        #############################################
        # VCA algorithm
        #############################################

        indice = np.zeros((R),dtype=int)
        A = np.zeros((R,R))
        A[-1,0] = 1

        for i in range(R):
            w = np.random.rand(R,1);   
            f = w - np.dot(A,np.dot(splin.pinv(A),w))
            f = f / splin.norm(f)
            
            v = np.dot(f.T,y)

            indice[i] = np.argmax(np.absolute(v))
            A[:,i] = y[:,indice[i]]        # same as x(:,indice(i))

        Ae = Yp[:,indice]

        self.Ae_ = Ae
        self.idx_ = indice
        self.Xp_ = Yp
        
        return self

    def transform(self, X, y=None, **kwargs):
        check_is_fitted(self)
        return self.Xp_[:,self.idx_]

    def _estimate_snr(self, X, X_mean_0, reduced_x) -> float:
        L, N = X.shape          # L number of bands (channels), N number of pixels
        p, N = reduced_x.shape  # p number of endmembers (reduced dimension)
        
        P_y     = np.sum(X**2)/float(N)
        P_x     = np.sum(reduced_x**2)/float(N) + np.sum(X_mean_0**2)
        return 10*np.log10( (P_x - p/L*P_y)/(P_y - P_x) )


In [ ]:
eea = VCA(4)
data = subject_data.reshape(subject_data.shape[0]*subject_data.shape[1]*subject_data.shape[2], subject_data.shape[3])
endmembers = eea.fit_transform(data)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(endmembers)
plt.show()

In [ ]:
y_m = np.mean(data,axis=1,keepdims=True)
y_m.shape

In [ ]:
Y_o = data - y_m
Y_o.shape

In [ ]:
R=3
N=Y_o.shape[1]
Ud  = splin.svd(np.dot(Y_o,Y_o.T)/float(N))[0][:,:R]

In [ ]:
# Ud = np.linalg.svd(np.dot(Y_o,Y_o.T)/N)[0][:,:R]

In [ ]:
x_p = np.dot(Ud.T, Y_o)

SNR = estimate_snr(data,y_m,x_p)
SNR